In [1]:
import numpy as np
import matplotlib.pyplot as plt
import skimage
from skimage import io, color
import skimage.color as skic
import skimage.filters as skif
import skimage.data as skid
import skimage.util as sku
from skimage import data, filters
from ipywidgets import widgets
from scipy.ndimage import convolve
from skimage import exposure
from skimage.filters import threshold_mean


%matplotlib inline

# Funcao para plotar uma imagem qualquer
def show(img):
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    ax.imshow(img, cmap=plt.cm.gray)
    ax.set_axis_off()
    plt.show()
    
def normaliza(img):
    p2, p98 = np.percentile(img, (0, 100))
    img_rescale = exposure.rescale_intensity(img, in_range=(0, 255))
    return img_rescale

def binariza(img):
    thresh = threshold_mean(out)
    binary = out > thresh
    show(binary)

def sobel3x3(img):
    
    sobel3x3y = np.array([[-1,-2,-1],
                          [0,0,0],
                          [1,2,1]])

    sobel3x3x = np.array([[-1,0,1],
                          [-2,0,2],
                          [-1,0,1]])
    
    
    sbimage = ndimage.convolve(ndimage.convolve(img, sobel3x3y, mode='mirror'), sobel3x3x, mode='mirror')
    
    return normaliza(sbimage)

def display_image_and_histogram(image, title):
    # pega parâmetros da imagem
    channels = image.shape[2] if len(image.shape) > 2 else 1
    is_colored = channels > 1

    # cria uma figura com espaço para a imagem e para o histograma
    figure, axes = plt.subplots(1, 2, figsize=(18, 5))
    image_axis, histogram_axis = axes

    # coloca a imagem em seu subplot
    image_axis.imshow(image, cmap='gray')
    image_axis.set_axis_off()
    image_axis.set_title('{} {}'.format(title, image.shape))

    # coloca o histograma em seu subplot
    if is_colored:
        channel_colors = ['red', 'green', 'blue']
        for i, c in enumerate(channel_colors):
            histogram_axis.hist(image[:, :, i].flatten(), bins=256, lw=1, color=c, histtype='step')
    else:
        histogram_axis.hist(image.ravel(), bins=256, lw=0, histtype='bar')
    histogram_axis.set_xlim(0, image.max())
    histogram_axis.set_yticks([])

    plt.show()
